In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)


In [2]:
episodes = pd.read_csv('Datasets/episodes.csv')
paintings = pd.read_csv('Datasets/paintings.csv')
dates = pd.read_fwf('Datasets/dates', header = None)


In [3]:
# Rename the existing column to 'combined'
dates.rename(columns={0: 'combined'}, inplace=True)

# Regex pattern to extract title and date (excluding notes)
pattern = r'^"([^"]+)" \(([^)]+)\)'

# Extract Title and Date  and Notes using regex pattern
dates[['Title', 'Date']] = dates['combined'].str.extract(pattern, expand=True)

# Extract the month from the Date string
dates['Month'] = dates['Date'].str.extract(r'(\w+) \d+, \d+')

# Drop the 'combined' column
dates.drop(columns=['combined'], inplace=True)

dates


,Title,Date,Month
0,A Walk in the Woods,"January 11, 1983",January
1,Mount McKinley,"January 11, 1983",January
2,Ebony Sunset,"January 18, 1983",January
3,Winter Mist,"January 25, 1983",January
4,Quiet Stream,"February 1, 1983",February
...,...,...,...
398,Evergreen Valley,"April 19, 1994",April
399,Balmy Beach,"April 26, 1994",April
400,Lake at the Ridge,"May 3, 1994",May
401,In the Midst of Winter,"May 10, 1994",May


In [13]:
# Convert episode column headers to title case
episodes.columns = episodes.columns.str.title()

# Rename title headers of episodes and paintings files to all match 'Title'
episodes.rename(columns={'TITLE': 'Title'}, inplace=True)
paintings.rename(columns={'painting_title': 'Title'}, inplace=True)

# Convert episode titles in columns 'Title' for each dataset to title case
episodes['Title'] = episodes['Title'].str.title()
dates['Title'] = dates['Title'].str.title()
paintings['Title'] = paintings['Title'].str.title()

paintings


,Unnamed: 0,painting_index,img_src,Title,season,episode,num_colors,youtube_src,colors,color_hex,Black_Gesso,Bright_Red,Burnt_Umber,Cadmium_Yellow,Dark_Sienna,Indian_Red,Indian_Yellow,Liquid_Black,Liquid_Clear,Midnight_Black,Phthalo_Blue,Phthalo_Green,Prussian_Blue,Sap_Green,Titanium_White,Van_Dyke_Brown,Yellow_Ochre,Alizarin_Crimson
0,1,282,https://www.twoinchbrush.com/images/painting28...,A Walk In The Woods,1,1,8,https://www.youtube.com/embed/oh5p5f5_-7A,"['Alizarin Crimson', 'Bright Red', 'Cadmium Ye...","['#4E1500', '#DB0000', '#FFEC00', '#102E3C', '...",0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
1,2,283,https://www.twoinchbrush.com/images/painting28...,Mt. Mckinley,1,2,8,https://www.youtube.com/embed/RInDWhYceLU,"['Alizarin Crimson', 'Bright Red', 'Cadmium Ye...","['#4E1500', '#DB0000', '#FFEC00', '#102E3C', '...",0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
2,3,284,https://www.twoinchbrush.com/images/painting28...,Ebony Sunset,1,3,9,https://www.youtube.com/embed/UOziR7PoVco,"['Alizarin Crimson', 'Black Gesso', 'Bright Re...","['#4E1500', '#000000', '#DB0000', '#FFEC00', '...",1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
3,4,285,https://www.twoinchbrush.com/images/painting28...,Winter Mist,1,4,3,https://www.youtube.com/embed/0pwoixRikn4,"['Prussian Blue', 'Titanium White', 'Van Dyke ...","['#021E44', '#FFFFFF', '#221B15']",0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0
4,5,286,https://www.twoinchbrush.com/images/painting28...,Quiet Stream,1,5,8,https://www.youtube.com/embed/DFSIQNjKRfk,"['Alizarin Crimson', 'Bright Red', 'Cadmium Ye...","['#4E1500', '#DB0000', '#FFEC00', '#102E3C', '...",0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,399,407,https://www.twoinchbrush.com/images/painting40...,Evergreen Valley,31,9,14,https://www.youtube.com/embed/mEU0stNfkxI,"['Alizarin Crimson', 'Bright Red', 'Cadmium Ye...","['#4E1500', '#DB0000', '#FFEC00', '#5F2E1F', '...",0,1,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1
399,400,408,https://www.twoinchbrush.com/images/painting40...,Balmy Beach,31,10,13,https://www.youtube.com/embed/kMgd6r6c4vE,"['Alizarin Crimson', 'Black Gesso', 'Bright Re...","['#4E1500', '#000000', '#DB0000', '#FFEC00', '...",1,1,0,1,1,0,1,0,1,1,1,1,0,0,1,1,1,1
400,401,409,https://www.twoinchbrush.com/images/painting40...,Lake At The Ridge,31,11,12,https://www.youtube.com/embed/8QWvzEQ69Kw,"['Alizarin Crimson', 'Bright Red', 'Cadmium Ye...","['#4E1500', '#DB0000', '#FFEC00', '#5F2E1F', '...",0,1,0,1,1,0,1,0,0,1,1,0,1,1,1,1,1,1
401,402,410,https://www.twoinchbrush.com/images/painting41...,In The Midst Of Winter,31,12,7,https://www.youtube.com/embed/qx2IsmrCs3c,"['Alizarin Crimson', 'Dark Sienna', 'Midnight ...","['#4E1500', '#5F2E1F', '#000000', '#0C0040', '...",0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,0,1
